In [ ]:
import pandas as pd
from typing import Any
import re

## CLEANING TASKS <br>
1.	Standardize column names (snake_case) <br>
2.	Strip whitespace from all string fields <br>
3.	Normalize missing values (NaN, empty string, “-”, “NA”, “n/a”) <br>
4.	Convert Amount In USD to numeric <br>
5.	Convert Date to datetime <br>
6.	Clean city names (Bangalore/Bengaluru, Bombay/Mumbai, etc.) <br>
7.	Extract year from Date <br>
8.	Standardize Investment Type (Seed/Seed Funding → Seed) <br>
9.	Split multiple investors into lists <br>
10.	Clean Startup Name (remove suffixes like “ Pvt Ltd”, “ Limited”) <br>



In [ ]:
df = pd.read_csv('/Users/olixstudios/Documents/workspace/Projects/indian-startups-transformations/data/starup.csv')
first_column_name = df.columns[0]
df = df.drop(columns=[first_column_name], errors="ignore")

In [ ]:
def clean_column_names(df: pd.DataFrame) -> pd.DataFrame:
    def _snake_case(s: Any) -> str:
        if not isinstance(s, str):
            s = str(s)
        s  = s.strip()
        s = s.replace(" ", "_")
        s = re.sub(r"[^\w\s]", "", s)  
        return s.lower()
    df =  df.copy()
    df.columns = [_snake_case(col) for col in df.columns]
    return df

In [ ]:
def normalize_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    MISSING_TOKENS = ["N/A", "n/a", "NA", "na", "-", "--", " ", ""]
    df = df.copy()
    df.replace(to_replace=MISSING_TOKENS, value=pd.NA, inplace=True)
    return df


In [ ]:
# df["amount"].fillna(0)

In [ ]:
# df["amount"] = df["amount"].str.replace(",", "")
# df["amount"] = pd.to_numeric(df["amount"], errors='coerce')
# df


In [ ]:
def clean_amount_column(df: pd.DataFrame, col: str = "amount") -> pd.DataFrame:
    df = df.copy()
    df[col] = df[col].astype(str).fillna("")
    #df[col] = df[col].str.replace(",", "")
    df[col] = df[col].str.strip()
    #df[col] = df[col].str.replace(r"[\$,€£¥]", "", regex=True)
    df[col] = df[col].str.replace(r"[^0-9\.\-]", '', regex=True).replace('', pd.NA)
    df[col] = pd.to_numeric(df[col], errors='coerce')
    return df


In [ ]:
def parse_dates(df: pd.DataFrame, col: str = "date") -> pd.DataFrame:
    df = df.copy()
    df[col] = pd.to_datetime(df[col], errors='coerce') 
    return df

In [ ]:
def date_analysis(df: pd.DataFrame, date_col: str = "date") -> pd.DataFrame:
    df = df.copy()
    df["year"] = df[date_col].dt.year
    df["date_missing"] = df[date_col].isna()
    return df

In [ ]:
print('orig cols:', df.columns.tolist())
df = clean_column_names(df)
print('clean cols:', df.columns.tolist())
df = normalize_missing_values(df)
print('null counts:', df.isna().sum().to_dict())
df = clean_amount_column(df, col='amount')   # or amount_in_usd
print('amount dtype:', df['amount'].dtype)
df = parse_dates(df, col='date')
print('date dtype:', df['date'].dtype)
print(df[['amount','date']].head().to_dict())

In [ ]:
import numpy as np
temp = df['type'].unique()
temp = pd.DataFrame(temp)
temp



In [ ]:
temp = df['type'].groupby(df['type']).size()
temp

In [ ]:
temp = df['type']
temp

In [ ]:
CANONICAL_PATTERNS = {
    r"^angel.*": "angel",
    r".*angel funding.*": "angel",
    r"^seed.*": "seed",
    r".*seed funding.*": "seed",
    r".*seed / angel.*": "seed",
    r"^pre[\s\-]?seed.*": "pre_seed",
    r"^pre[\s\-]?series a.*": "pre_seed",   # or "pre_series_a"
    r"^series [a-z0-9].*": "series",
    r".*venture.*": "venture",
    r".*private equity.*": "private_equity",
    r".*debt.*": "debt",
    r".*equity.*": "equity",
    r".*mezzanine.*": "mezzanine",
    r".*m&a.*": "ma",
    r".*bridge.*": "bridge",
    r".*secondary market.*": "secondary_market",
    r".*in progress.*": "in_progress",
    r".*unspecified.*": "unspecified",
}

def canonical_investment_type(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.lower().str.strip()

    out = s.replace(CANONICAL_PATTERNS, regex=True)
    out = out.where(out.isin(set(CANONICAL_PATTERNS.values())), other="other")

    return out


In [ ]:
df["type"].value_counts()

In [ ]:

df["type_canonical"] = canonical_investment_type(df["type"])
df["type_canonical"].value_counts()

In [ ]:
SPLIT_PATTERN = r",\s*|;\s*|\s+&\s+"
def split_investors(s: pd.Series) -> pd.Series:
    s = s.astype(str).str.strip()
    out = s.fillna("").str.split(SPLIT_PATTERN)
    return out

df['investor_list'] = split_investors(df['investor'])
df['investor_count'] = df['investor_list'].apply(len)
df

In [ ]:
temp = df['startup'].nunique() == len(df['startup'])
print(df['startup'].nunique())
print(len(df['startup']))
temp

In [ ]:
gp = df['startup'].groupby(df['startup']).size().sort_values(ascending=False)
pd.DataFrame(gp)
gp

In [ ]:
SUFFIX_PATTERN = r"""
    \s+(
        pvt\.?\s*ltd      |   # Pvt Ltd / Pvt. Ltd
        private\s*limited |   # Private Limited
        ltd\.?            |   # Ltd / Ltd.
        limited           |   # Limited
        inc\.?            |   # Inc / Inc.
        incorporated      |
        corp\.?           |
        corporation
    )\s*$
"""    


def clean_startup_name(name: str) -> str:
    if not isinstance(name, str):
        return name

    # Normalize spaces and case
    s = name.strip()
    s = re.sub(SUFFIX_PATTERN, "", s, flags=re.IGNORECASE | re.VERBOSE)
    return s.strip()
df["startup_clean"] = df["startup"].apply(clean_startup_name)
df